# Get Out The Vote! Python Code Base

## - KTWT Consulting

In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import dateutil.parser as dp
import os
import re
import warnings
%matplotlib inline 

STATION_GROUP = ['STATION'] #['C/A', 'UNIT', 'STATION']
TURN_GROUP = STATION_GROUP + ['SCP']

In [ ]:
def ChooseTurnstile(station):
    a = turn.loc[turn["STATION"] == station]
    return a

def ChooseTurnstile2015(station, season):
    a = turn.loc[turn["STATION"] == station]
    a = a.rename(columns = {'EXITS                                                               ':'EXITS'})
    a['DATE'] = pd.to_datetime(a['DATE'])
#     a = a.groupby(by=['STATION','DATE','SCP'])['EXITS'].sum().reset_index()
    a = a.groupby(by=['C/A','UNIT','STATION','SCP','DATE','TIME'])['EXITS'].sum().reset_index()
    a.loc[:, 'NET_EXITS'] = a['EXITS'] - a['EXITS'].shift(1)
    a = a[a['NET_EXITS'] > 0]
    a = a[a['NET_EXITS'] < 10000]
    a.loc[:, 'WEEKDAY'] = a['DATE'].map(lambda x: x.isoweekday())
    a = a.groupby(by=['STATION','DATE','WEEKDAY'])['NET_EXITS'].sum().reset_index()
    if season == 'summer':
        sumdat = a[(a['DATE'] > '2015-06-26') & (a['DATE'] < '2015-07-24')].reset_index()
        return sumdat
    else:
        falldat = a[(a['DATE'] > '2015-10-03') & (a['DATE'] < '2015-12-01')].reset_index()
        return falldat

def ChooseTurnstile2016(station, season):
    a = turn.loc[turn["STATION"] == station]
    a = a.rename(columns = {'EXITS                                                               ':'EXITS'})
    a['DATE'] = pd.to_datetime(a['DATE'])
#     a = a.groupby(by=['STATION','DATE','SCP'])['EXITS'].sum().reset_index()
    a = a.groupby(by=['C/A','UNIT','STATION','SCP','DATE','TIME'])['EXITS'].sum().reset_index()
    a.loc[:, 'NET_EXITS'] = a['EXITS'] - a['EXITS'].shift(1)
    a = a[a['NET_EXITS'] > 0]
    a = a[a['NET_EXITS'] < 5000]
#     print(a)
    a.loc[:, 'WEEKDAY'] = a['DATE'].map(lambda x: x.isoweekday())
    a = a.groupby(by=['STATION','DATE','WEEKDAY'])['NET_EXITS'].sum().reset_index()
    if season == 'summer':
        sumdat = a[(a['DATE'] > '2016-07-02') & (a['DATE'] < '2016-07-23')].reset_index()
        return sumdat
    else:
        falldat = a[(a['DATE'] > '2016-10-08') & (a['DATE'] < '2016-12-01')].reset_index()
        return falldat

In [ ]:
def plotStations(summer, fall, station, yaxisV = 55000):
    #This is the code for plotting
    plt.scatter(summer['day'], summer['summer exits'])
    plt.scatter(fall['day'], fall['fall exits'])
    plt.ylim(0, yaxisV)
    plt.xlabel('Day (Count Starting from 1st Monday of the Month)')
    plt.ylabel('Number of People Exiting') #, fontsize=40)
    plt.title('Passenger Traffic At "{0:s}" Station in Summer and Fall'.format(station))
    plt.legend(('July 2016', 'October 2016'), loc = 'lower right')
    return plt.show()

In [ ]:
def Alt_plotStations(summer, fall, station, yaxisV = 55000):
    #This is the code for plotting
    s = [150*1 for n in range(len(list(summer['day'])))]
    plt.scatter(summer['day'], summer['summer exits'], s = s)
    plt.scatter(fall['day'], fall['fall exits'], s = s)
    plt.ylim(0, yaxisV)
    plt.ylabel('Number of People Exiting', fontsize=20)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.title('Passenger Traffic At "{0:s}" Station in Summer and Fall'.format(station), fontsize=22, fontweight = "bold")
    plt.legend(('July 2016', 'October 2016'), loc = 'lower right', fontsize=16)
    return None #plt.show()

In [ ]:
def ExitStation(new_dogDay):
    #This creates dataframe but does not plot
    newNames = ['index', 'c/a', 'unit', 'scp', 'station', 'linename', 'division', 'date', 'time', 'desc', 'entries', 'exits']
    new_dogDay.columns = newNames
    new_dogDay['corrected'] = new_dogDay.groupby(['c/a','unit', 'scp', 'date'])['exits'].transform(pd.Series.diff)
    new_dogDay_exits = new_dogDay[['c/a', 'unit', 'scp', 'date', 'exits']]
    new_dogDay_exits['corrected'] = new_dogDay_exits.groupby(['c/a','unit', 'scp', 'date'])['exits'].transform(pd.Series.diff)
    ndD = new_dogDay_exits[['date', 'corrected']]
    ndD_combined = ndD.groupby('date').sum()
    ndD = ndD_combined['corrected']
    ndD = ndD.reset_index()
    ndD['date'] = pd.to_datetime(ndD['date'])
    ndD.rename(columns = {'corrected' : 'exits'}, inplace = True)
    summer = (ndD['date'] > dp.parse('2016-07-03')) & (ndD['date'] <= dp.parse('2016-08-30'))
    fall = (ndD['date'] > dp.parse('2016-10-02')) & (ndD['date'] <= dp.parse('2016-12-30'))
    ndDj = ndD[summer]
    ndDo = ndD[fall]
    ndDj.rename(columns = {'exits' : 'summer exits'}, inplace = True)
    ndDo.rename(columns = {'exits' : 'fall exits'}, inplace = True)
    day = list(range(1, 27))
    ndDj['day'] = day
    ndDo['day'] = day
    return ndDj, ndDo

In [ ]:
def ExitStationP(dogDay, V):
    #This creates the dataframe and plots
    new_dogDay = ChooseTurnstile(dogDay)
    
    newNames = ['index', 'c/a', 'unit', 'scp', 'station', 'linename', 'division', 'date', 'time', 'desc', 'entries',
                'exits']
    new_dogDay.columns = newNames
    
    new_dogDay['corrected'] = new_dogDay.groupby(['c/a','unit', 'scp', 'date'])['exits'].transform(pd.Series.diff)
    
    new_dogDay_exits = new_dogDay[['c/a', 'unit', 'scp', 'date', 'exits']]
    
    new_dogDay_exits['corrected'] = new_dogDay_exits.groupby(['c/a','unit', 'scp', 'date'])['exits'].transform(pd.Series.diff)
    
    ndD = new_dogDay_exits[['date', 'corrected']]
    
    ndD_combined = ndD.groupby('date').sum()
    
    ndD = ndD_combined['corrected']
    ndD = ndD.reset_index()
    
    ndD['date'] = pd.to_datetime(ndD['date'])
    ndD.rename(columns = {'corrected' : 'exits'}, inplace = True)
    
    summer = (ndD['date'] > dp.parse('2016-07-03')) & (ndD['date'] <= dp.parse('2016-08-30'))
    fall = (ndD['date'] > dp.parse('2016-10-02')) & (ndD['date'] <= dp.parse('2016-12-30'))
    ndDj = ndD[summer]
    ndDo = ndD[fall]
    
    ndDj.rename(columns = {'exits' : 'summer exits'}, inplace = True)
    ndDo.rename(columns = {'exits' : 'fall exits'}, inplace = True)
    
    day = list(range(1, 27))
    ndDj['day'] = day
    ndDo['day'] = day
    
    return plotStations(ndDj, ndDo, dogDay, V)

In [ ]:
#Bar plot functions by station, by season
def GraphStation(station, season):
    """
    Output: Line plot of one station
    """
    raw_data = ChooseTurnstile2015(station, season)
#     b = raw_data.iloc[0]
    plt.plot(raw_data.index, raw_data['NET_EXITS'])
    
def GraphStationBar(station, season):
    raw_data = ChooseTurnstile2015(station, season)
#     b = raw_data.iloc[0]
    plt.bar(raw_data.index, raw_data['NET_EXITS'])
    
def Graphing2015Line(station):
    """
    Output: Line plot of one station by season
    """
    raw_data1 = ChooseTurnstile2015(station, "fall")
    raw_data2 = ChooseTurnstile2015(station, "summer")
    plt.figure(figsize = (15, 6))
    plt.plot(raw_data1.index, raw_data1['NET_EXITS'], 'b', label="Fall 2015")
    plt.plot(raw_data2.index, raw_data2['NET_EXITS'], 'y', label="Summer 2015")
    plt.ylabel(r"Total Exits", fontsize = 14)
    plt.xlabel(r"Days Surveyed", fontsize = 14)
    plt.title("2015 " + station + " Turnstile Traffic by Season", fontsize = 16, fontweight="bold")
    plt.legend(loc='upper left')
#     difference = raw_data1['NET_EXITS'].reset_index(drop=True) - raw_data2['NET_EXITS'].reset_index(drop=True)
#     plt.plot(difference.index, difference['NET_EXITS'])
#     print(difference)
    plt.show()

def Graphing2016Line(station):
    raw_data1 = ChooseTurnstile2016(station, "fall")
    raw_data2 = ChooseTurnstile2016(station, "summer")
#     print(raw_data1)
    plt.figure(figsize = (15, 6))
    plt.plot(raw_data1.index, raw_data1['NET_EXITS'], 'b', label="Fall 2016")
    plt.plot(raw_data2.index, raw_data2['NET_EXITS'], 'y', label="Summer 2016")
    plt.ylabel(r"Total Exits", fontsize = 14)
    plt.xlabel(r"Days Surveyed", fontsize = 14)
    plt.title("2016 " + station + " Turnstile Traffic by Season", fontsize = 16, fontweight="bold")
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
#Bar plot functions by station, by season, and comparing differences in ridership
def Graphing2015BarbyDay(station):
    raw_data1 = ChooseTurnstile2015(station, "fall")
    raw_data2 = ChooseTurnstile2015(station, "summer")
    data_sets = 27
    fig, ax = plt.subplots()
    index = np.arange(data_sets)
    bar_width = 0.35   
    opacity = 0.4
    error_config = {'ecolor': '0.3'}
    rects1 = plt.bar(raw_data1['WEEKDAY'], raw_data1['NET_EXITS'], bar_width,
                 alpha=opacity,
                 color='b',
                 label='Fall')
    rects2 = plt.bar(raw_data2['WEEKDAY'], raw_data2['NET_EXITS'], bar_width,
                 alpha=opacity,
                 color='r',
                 label='Summer')

    plt.xlabel('Days Surveyed')
    plt.ylabel('Total Exits')
    plt.title('2015 Turnstile Data for ' + station + ' Subway Station')
    plt.legend()
    plt.tight_layout()
    plt.show()

def Graphing2016BarbyDay(station):
    raw_data1 = ChooseTurnstile2016(station, "fall")
    raw_data2 = ChooseTurnstile2016(station, "summer")
    data_sets = 21
    fig, ax = plt.subplots()
    index = np.arange(data_sets)
    bar_width = 0.30   
    opacity = 0.4
    error_config = {'ecolor': '0.3'}
    rects1 = plt.bar(raw_data1['WEEKDAY'], raw_data1['NET_EXITS'], bar_width,
                 alpha=opacity,
                 color='b',
                 label='Fall')
    rects2 = plt.bar(raw_data2['WEEKDAY'], raw_data2['NET_EXITS'], bar_width,
                 alpha=opacity,
                 color='r',
                 label='Summer')
    plt.xlabel('Days Surveyed')
    plt.ylabel('Total Exits')
    plt.title('2016 Turnstile Data for ' + station + ' Subway Station')
    plt.legend()
    plt.tight_layout()
    plt.show()

def Graphing2016BarbySpread(station):
    raw_data1 = ChooseTurnstile2016(station, "fall")
    raw_data2 = ChooseTurnstile2016(station, "summer")
    data_sets = 21
    fig, ax = plt.subplots()
    index = np.arange(data_sets)
    bar_width = 0.35   
    opacity = 0.4
    error_config = {'ecolor': '0.3'}
    rects1 = plt.bar(raw_data1.index, raw_data1['NET_EXITS'], bar_width,
                 alpha=opacity,
                 color='b',
                 label='Fall')
    rects2 = plt.bar(raw_data2.index, raw_data2['NET_EXITS'], bar_width,
                 alpha=opacity,
                 color='r',
                 label='Summer')

    plt.xlabel('Days Surveyed')
    plt.ylabel('Total Exits')
    plt.title('2016 Turnstile Data for ' + station + ' Subway Station')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
dir1 = r'/Users/walterkerr/metis/sf17_ds6/week1/Benson_Presentation/Benson_Challenges'
for files in os.walk(dir1):
    for file in files:
        print(file)
        #filenames += [file]

try:
    del turn
except:
    pass

filenames = ['turnstile_150704.txt', 'turnstile_150711.txt', 'turnstile_150718.txt', 'turnstile_150725.txt', 'turnstile_151003.txt', 'turnstile_151010.txt', 'turnstile_151017.txt', 'turnstile_151024.txt', 'turnstile_151031.txt', 'turnstile_160702.txt', 'turnstile_160709.txt', 'turnstile_160716.txt', 'turnstile_160723.txt', 'turnstile_160730.txt', 'turnstile_161001.txt', 'turnstile_161008.txt', 'turnstile_161015.txt', 'turnstile_161022.txt', 'turnstile_161029.txt']
for fn in filenames:
    temp = pd.read_csv(fn)
    try:
        turn = turn.append(temp)
    except:
        turn = temp.copy()
turn = turn.reset_index()

In [ ]:
#Old data cleaning codes
shape = turn.shape
turn.columns = turn.columns.str.strip()
turn.loc[:, 'd'] = turn['DATE'].map(lambda x: pd.to_datetime(x))
turn.loc[:, 'ts'] = turn['DATE'].str.cat(turn['TIME'], sep = " ").map(lambda x: pd.to_datetime(x))
turn.loc[:, 'hr'] = turn['ts'].dt.hour
turn.loc[:, 'wday_ch'] = turn.loc[:, 'd'].map(lambda x: x.strftime("%A"))
turn.loc[:, 'wday_num'] = turn.loc[:, 'd'].map(lambda x: x.isoweekday())
turn.loc[:, 'wk'] = turn.loc[:, 'd'].map(lambda x: x.isocalendar()[1])
#turn['d'].value_counts()
turn.loc[turn['d'].isin(pd.date_range(dt.datetime(2015, 6, 1), dt.datetime(2015, 8, 1))), 'timeslot'] = "Summer2015"
turn.loc[turn['d'].isin(pd.date_range(dt.datetime(2015, 9, 1), dt.datetime(2015, 11, 1))), 'timeslot'] = "Fall2015"
turn.loc[turn['d'].isin(pd.date_range(dt.datetime(2016, 6, 1), dt.datetime(2016, 8, 1))), 'timeslot'] = "Summer2016"
turn.loc[turn['d'].isin(pd.date_range(dt.datetime(2016, 9, 1), dt.datetime(2016, 11, 1))), 'timeslot'] = "Fall2016"
turn.sort_values(TURN_GROUP + ['ts'], inplace = True)
turn.loc[:, 'net_enter'] = turn['ENTRIES'] - turn.groupby(TURN_GROUP)['ENTRIES'].shift(1)
turn.loc[:, 'net_exits'] = turn['EXITS'] - turn.groupby(TURN_GROUP)['EXITS'].shift(1)
turn.loc[:, 'tot_traffic'] = turn['net_enter'] + turn['net_exits']
#turn.loc[:, 'net_traffic'] = turn['net_enter'] - turn['net_exits']
turn = turn[turn['DESC'] == 'REGULAR']
turn.loc[turn['net_enter'] < 0, 'net_enter'] = np.nan
turn.loc[turn['net_exits'] < 0, 'net_exits'] = np.nan
#turn['net_exits'].hist(); turn.loc[turn['net_exits'] > 20000, 'd'].value_counts()
turn.loc[turn['net_exits'] > 20000, 'net_exits'] = np.na

In [ ]:
#Generate single station DataFrame 
ChooseTurnstile2016('CHAMBERS ST','fall')

In [ ]:
#Generate scatter plot 
ExitStationP('33 ST-RAWSON ST', 15000)

In [ ]:
#Generate alternative scatter plot 
plt.figure(figsize = (15, 10))
ax1 = plt.subplot(2, 1, 1)
plt.subplot(2, 1, 1)
Alt_ExitStationP('33 ST-RAWSON ST', 15000)
plt.setp(ax1.get_xticklabels(), visible=False)
plt.subplot(2, 1, 2, sharex=ax1) #, sharey=ax1)
plt.xlabel('Day (Count Starting from 1st Monday of the Month)', fontsize=20)
Alt_ExitStationP('GREENPOINT AV', 7000)
plt.show()

In [ ]:
#Generate line plot
columbia_summer2015 = GraphStation('116 ST','summer')

In [ ]:
#Generate line plot by season
Graphing2016Line("116 ST")

In [ ]:
#Generate bar plot that compares differences in ridership
Graphing2015Bar('33 ST/RAWSON ST')